In [130]:
import pandas as pd
import numpy as np
import csv
import re #regular expression
import nltk
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline
from sklearn.externals import joblib
from sklearn import linear_model
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB 
from sklearn.svm import SVC

# Importing Data set

In [2]:
data_train=pd.read_csv('Data/dataset/train.csv')

In [3]:
data_test=pd.read_csv('Data/dataset/test.csv')

# Preprocessing

In [4]:
data_test=data_test.fillna(' ')
data_train=data_train.fillna(' ')
#data_train['total']=data_train['original_text']+' '+data_train['original_author']+' '+data_train['retweet_count']+' '+data_train['lang']
#data_test['total']=data_test['original_text']+' '+data_test['original_author']+' '+data_test['retweet_count']+' '+data_test['lang']

In [297]:
stop_words = stopwords.words('english')
lemmatizer=WordNetLemmatizer()

In [298]:
stop_words.remove('no')
stop_words.remove('not')
stop_words.remove("didn't")

In [299]:
stop_words

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [300]:
def cleanText(text):
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', text) # remove URLs
    text = re.sub('@[^\s]+', 'AT_USER', text) # remove usernames
    text = re.sub(r'#([^\s]+)', r'\1', text) # remove the # in #hashtag
    text = re.sub(r'@[A-Za-z0-9]','',text)
    text = re.sub(r'[0-9]','',text)
    text = re.sub(r'[^\w\s]','',text)
    return text

In [301]:
def preProcess(data):
    lemmatizer=WordNetLemmatizer()
    for index,row in data.iterrows():
        filter_sentence = ''

        sentence = row['original_text']
        #print("1 - "+sentence)
        sentence = cleanText(sentence) #cleaning
        #print("2 - "+sentence)
        words = nltk.word_tokenize(sentence) #tokenization
        words = [w for w in words if not w in stop_words]  #stopwords removal
        #print(words)
        for word in words:
            filter_sentence = filter_sentence + ' ' + str(lemmatizer.lemmatize(word)).lower() # lemmatize and convert to lower-case
        #print("4 - "+filter_sentence)   
        data.loc[index,'original_text'] = filter_sentence
    return data

In [302]:
data_train=preProcess(data_train)
train = data_train[['original_text','sentiment_class']]

# Applying NLP Techniques

In [303]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.70, min_df=2, max_features=1000, stop_words=stop_words)
# TF-IDF feature matrix
tfidf = tfidf_vectorizer.fit_transform(train['original_text'])

In [322]:
#split in samples
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
X_train1, X_train2, y_train1, y_train2 = train_test_split(tfidf, train['sentiment_class'], random_state=2,stratify= train['sentiment_class'])

In [323]:
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [324]:
lin_clf = SVC(kernel='linear',decision_function_shape='ovr')
#lin_clf = svm.SVC(decision_function_shape='ovo')
#lin_clf = LogisticRegression(multi_class='ovr',class_weight='balanced')
#lin_clf = RandomForestClassifier(max_depth=2, random_state=0)
#lin_clf = MLPClassifier(random_state=1, max_iter=300).fit(X_train1, y_train1)
#lin_clf = GradientBoostingClassifier()
#lin_clf = SGDClassifier()
lin_clf.fit(X_train1, y_train1)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [325]:
pred1 = lin_clf.predict(X_train1)
pred2 = lin_clf.predict(X_train2)

In [326]:
(pred1[pred1==1].size/pred1.size)*100

4.740313272877164

In [327]:
score1=f1_score(y_train1, pred1, average='weighted')*100
score2=f1_score(y_train2, pred2, average='weighted')*100

In [328]:
score1

52.89721333866735

In [329]:
score2

38.047378506081614

In [330]:
pred1[pred1==1].size

115

# Predict for Test Data

In [331]:
data_test=preProcess(data_test)
test = data_test[['original_text']]

In [332]:
test

,original_text
0,yeah cooked potato year old mean threw bag sp...
1,happy mother day mum stepmums grandmother dad...
2,love people uk however awoke saw mothersday t...
3,happy st birthday happy mother day mum cant s...
4,happy mother day wonderful mother world live ...
...,...
1382,happy mother day crazy bitch im sorry covid b...
1383,mum cant get bingo brought bingo top prize in...
1384,despite bullshit today one favourite day lonn...
1385,happy mother day firstly queen daniele_sulliv...


In [333]:
tfidf_test = tfidf_vectorizer.fit_transform(test['original_text'])
pred_test = lin_clf.predict(tfidf_test)

In [335]:
pd.DataFrame(pred_test).to_csv("tweet_sentiments.csv",header="sentiment_class",index=False)